<a href="https://colab.research.google.com/github/Aastha-tembhare/Image_denoising-24/blob/main/Final_Image_denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import os
import glob
from sklearn.model_selection import train_test_split
import shutil
from PIL import Image
import torch.nn.functional as F
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.augmentations as augmentations
from tqdm import tqdm


In [ ]:
# SCPA Block
class SCPA(nn.Module):
    def __init__(model,in_ch,out_ch):
        super(SCPA,model).__init__()
        model.conv1_branch1 = nn.Conv2d(in_ch, out_ch, k_size=1, strd=1, pad=0)
        model.conv2_branch1 = nn.Conv2d(in_ch, out_ch, k_size=3, strd=1, pad=1)
        model.sigmoid = nn.Sigmoid()
        model.conv1_branch2 = nn.Conv2d(in_ch, out_ch, k_size=1, strd=1, pad=0)
        model.conv2_branch2 = nn.Conv2d(in_ch, out_ch, k_size=1, strd=1, pad=0)
        model.conv3_branch2 = nn.Conv2d(in_ch, out_ch, k_size=3, strd=1, pad=1)
        model.conv4_branch2 = nn.Conv2d(in_ch, out_ch, k_size=3, strd=1, pad=1)
        model.final_conv =    nn.Conv2d(in_ch, out_ch, k_size=1, strd=1, pad=0)
    def forward(model,input_tensor):
        # branch 1
        branch1 = model.conv1_branch1(input_tensor)
        branch1 = model.conv2_branch1(branch1)
        # branch 2
        branch2 = model.conv1_branch2(input_tensor)
        branch2a = model.conv2_branch2(branch2)
        branch2a = model.sigmoid(branch2a)
        branch2b = model.conv3_branch2(branch2)
        branch2 = branch2a*branch2b
        branch2 = model.conv4_branch2(branch2)

        #combining branch 1 and branch 2
        output = branch2 + branch1

        #final convolutional layer and add it to the orignal input
        final_conv = model.final_conv(output)
        SCPA_output = final_conv + input_tensor
        return SCPA_output



In [ ]:
# CoordConv Bloack
class CoordConv(nn.Module):
    def __init__(model, in_ch, out_ch):
        super(CoordConv, model).__init__()
        model.conv = nn.Conv2d(in_ch+2, out_ch, k_size=3, pad=1)

    def forward(model, input_tensor):
        batch_size, _, height, width = input_tensor.size()
        input_tensorinput_tensor = torch.arange(width).repeat(height, 1)
        yy = torch.arange(height).view(-1, 1).repeat(1, width)
        input_tensorinput_tensor = input_tensorinput_tensor.float() / (width - 1)
        yy = yy.float() / (height - 1)
        input_tensorinput_tensor = input_tensorinput_tensor.repeat(batch_size, 1, 1).unsqueeze(1)
        yy = yy.repeat(batch_size, 1, 1).unsqueeze(1)
        if input_tensor.is_cuda:
            input_tensorinput_tensor = input_tensorinput_tensor.cuda()
            yy = yy.cuda()
        # adding input_tensor and y cordinate to RGB channel
        input_tensor = torch.cat([input_tensor, input_tensorinput_tensor, yy], dim=1)
        input_tensor = model.conv(input_tensor)
        return input_tensor



In [ ]:
class InvResBlock(nn.Module):
    def __init__(model, in_ch, mid_channels, out_ch):
        super(InvResBlock, model).__init__()
        model.conv1 = nn.Conv2d(in_ch, mid_channels, k_size=1, pad=0)
        model.bn1 = nn.BatchNorm2d(mid_channels)
        model.relu = nn.ReLU()
        model.conv2 = nn.Conv2d(mid_channels, mid_channels, k_size=3, pad=1)
        model.bn2 = nn.BatchNorm2d(mid_channels)
        model.conv3 = nn.Conv2d(mid_channels, out_ch, k_size=1, pad=0)
        model.bn3 = nn.BatchNorm2d(out_ch)

    def forward(model, input_tensor):
        identity = input_tensor

        out = model.conv1(input_tensor)
        out = model.bn1(out)
        out = model.relu(out)

        out = model.conv2(out)
        out = model.bn2(out)
        out = model.relu(out)

        out = model.conv3(out)
        out = model.bn3(out)

        out += identity

        out = model.relu(out)

        return out


In [ ]:
class AttBlock(nn.Module):
    def __init__(model, in_ch):
        super(AttBlock, model).__init__()
        model.conv1 = nn.Conv2d(in_ch, 1, k_size=1)
        model.sigmoid = nn.Sigmoid()

    def forward(model, input_tensor):
        # Calculate attention weights
        attn_weights = model.conv1(input_tensor)
        attn_weights = model.sigmoid(attn_weights)
        # Apply attention to input features
        input_tensor_attention = input_tensor * attn_weights
        return input_tensor_attention


In [ ]:
def double_conv(in_c,out_c):
    conv = nn.Sequential(
    nn.Conv2d(in_c,out_c,k_size=3,pad =1), # pad is added on my own
    nn.ReLU(inplace=True),
    nn.Conv2d(out_c,out_c,k_size=3,pad =1 ),
    nn.ReLU(inplace=True)
    )
    return conv
def crop_img(tensor, target_tensor):
    _, _, h, w = target_tensor.size()
    return tensor[:, :, :h, :w]

class Unet(nn.Module):
    def __init__(model):
        super(Unet, model).__init__()
        model.mainput_tensor_pool_2input_tensor2 = nn.Mainput_tensorPool2d(k_size=2, strd=2)
        model.down_conv_1 = double_conv(3, 64)
        model.down_conv_2 = double_conv(64, 128)
        model.down_conv_3 = double_conv(128, 256)
        model.down_conv_4 = double_conv(256, 512)
        model.down_conv_5 = double_conv(512, 1024)

        model.up_trans_1 = nn.ConvTranspose2d(in_ch=1024, out_ch=512, k_size=2, strd=2)
        model.up_conv_1 = double_conv(1024, 512)
        model.up_trans_2 = nn.ConvTranspose2d(in_ch=512, out_ch=256, k_size=2, strd=2)
        model.up_conv_2 = double_conv(512, 256)
        model.up_trans_3 = nn.ConvTranspose2d(in_ch=256, out_ch=128, k_size=2, strd=2)
        model.up_conv_3 = double_conv(256, 128)
        model.up_trans_4 = nn.ConvTranspose2d(in_ch=128, out_ch=64, k_size=2, strd=2)
        model.up_conv_4 = double_conv(128, 64)
        model.out = nn.Conv2d(in_ch=64, out_ch=3, k_size=1)

    def forward(model, img):
        input_tensor1 = model.down_conv_1(img)
        input_tensor2 = model.mainput_tensor_pool_2input_tensor2(input_tensor1)  # skip
        input_tensor3 = model.down_conv_2(input_tensor2)
        input_tensor4 = model.mainput_tensor_pool_2input_tensor2(input_tensor3)  # skip
        input_tensor5 = model.down_conv_3(input_tensor4)
        input_tensor6 = model.mainput_tensor_pool_2input_tensor2(input_tensor5)  # skip
        input_tensor7 = model.down_conv_4(input_tensor6)
        input_tensor8 = model.mainput_tensor_pool_2input_tensor2(input_tensor7)  # skip
        input_tensor9 = model.down_conv_5(input_tensor8)

        input_tensor = model.up_trans_1(input_tensor9)
        y = crop_img(input_tensor7, input_tensor)
        input_tensor = model.up_conv_1(torch.cat([input_tensor, y], 1))

        input_tensor = model.up_trans_2(input_tensor)
        y = crop_img(input_tensor5, input_tensor)
        input_tensor = model.up_conv_2(torch.cat([input_tensor, y], 1))

        input_tensor = model.up_trans_3(input_tensor)
        y = crop_img(input_tensor3, input_tensor)
        input_tensor = model.up_conv_3(torch.cat([input_tensor, y], 1))

        input_tensor = model.up_trans_4(input_tensor)
        y = crop_img(input_tensor1, input_tensor)
        input_tensor = model.up_conv_4(torch.cat([input_tensor, y], 1))

        input_tensor = model.out(input_tensor)
        return input_tensor


In [ ]:
# Denoising branch
'''
convolutional block -> 4 inv residual block -> attention block -> convolution block
'''
class DenoiseBranch(nn.Module):
    def __init__(model,in_ch=3,out_ch=3):
        super(DenoiseBranch,model).__init__()
        model.conv_1 = nn.Conv2d(in_ch=in_ch,out_ch =256,k_size = 3,pad =1)
        model.inv_1 = InvResBlock(in_ch=256, mid_channels=128, out_ch=256)
        model.inv_2 = InvResBlock(in_ch=256, mid_channels=128, out_ch=256)
        model.inv_3 = InvResBlock(in_ch=256, mid_channels=128, out_ch=256)
        model.inv_4 = InvResBlock(in_ch=256, mid_channels=128, out_ch=256)
        model.attention = AttBlock(in_ch = 256)
        model.conv_2 = nn.Conv2d(in_ch=256,out_ch = 3,k_size = 3,pad =1)

    def forward(model,input_tensor):
        input_tensor = model.conv_1(input_tensor)
        input_tensor = model.inv_1(input_tensor)
        input_tensor = model.inv_2(input_tensor)
        input_tensor = model.inv_3(input_tensor)
        input_tensor = model.inv_4(input_tensor)
        input_tensor = model.attention(input_tensor)
        input_tensor = model.conv_2(input_tensor)
        return input_tensor


In [ ]:
class SCPA_Branch(nn.Module):
    def __init__(model,in_ch=3,out_ch=3):
        super(SCPA_Branch,model).__init__()
        model.coord_layer = CoordConv(in_ch =3,out_ch = 5)
        model.SCPA_1 = SCPA(in_ch = 5, out_ch =5)
        model.SCPA_2 = SCPA(in_ch = 5, out_ch =5)
        model.SCPA_3 = SCPA(in_ch = 5, out_ch =5)
        model.SCPA_4 = SCPA(in_ch = 5, out_ch =5)
        model.SCPA_5 = SCPA(in_ch = 5, out_ch =5)
        model.conv_layer =nn.Conv2d(in_ch=5,out_ch =3,k_size = 3,pad =1)

    def forward(model,input_tensor):
        input_tensor = model.coord_layer(input_tensor)
        input_tensor = model.SCPA_1(input_tensor)
        input_tensor = model.SCPA_2(input_tensor)
        input_tensor = model.SCPA_3(input_tensor)
        input_tensor = model.SCPA_4(input_tensor)
        input_tensor = model.SCPA_5(input_tensor)
        input_tensor = model.conv_layer(input_tensor)
        return input_tensor


In [ ]:
class LowLightModel(nn.Module):
    def __init__(model,in_ch=3,out_ch=3):
        super(LowLightModel,model).__init__()
        model.SCPA_branch = SCPA_Branch(in_ch=3,out_ch=3)
        model.Denoiser = DenoiseBranch(in_ch=3,out_ch=3)
        model.Unet = Unet()
        model.Conv = nn.Conv2d(in_ch=3,out_ch =3,k_size = 3,pad =1)

    def forward(model,input_tensor):
        # denoiser branch
        denoised = model.Denoiser(input_tensor)

        #SCPA branch
        SCPA = model.SCPA_branch(input_tensor)
        Unet_input = SCPA + input_tensor
        Unet_output = model.Unet(Unet_input)
        # pad
        diff = Unet_input.size(3) - Unet_output.size(3)
        pad_left = diff // 2
        pad_right = diff - pad_left

        # Pad Unet_output
        Unet_output = F.pad(Unet_output, (pad_left, pad_right, 0, 0))

        output = model.Conv(Unet_output)
        final_output = output + denoised
        return final_output


In [ ]:
class LowLightDataset(Dataset):
    def __init__(model, low_light_dir, bright_light_dir, augmentation=None):
        model.low_light_dir = low_light_dir
        model.bright_light_dir = bright_light_dir
        model.low_imgs = sorted(os.listdir(low_light_dir))
        model.high_imgs = sorted(os.listdir(bright_light_dir))
        model.augmentation = augmentation

    def __len__(model):
        return len(model.low_imgs)

    def __getitem__(model, idinput_tensor):
        low_img_path = os.path.join(model.low_light_dir, model.low_imgs[idinput_tensor])
        high_img_path = os.path.join(model.bright_light_dir, model.high_imgs[idinput_tensor])
        low_img = Image.open(low_img_path).convert("RGB")
        high_img = Image.open(high_img_path).convert("RGB")

        if model.augmentation:
            low_img = model.augmentation(low_img)
            high_img = model.augmentation(high_img)

        return low_img, high_img

augmentation = augmentations.Compose([
    augmentations.ToTensor(),
])


In [ ]:
def psnr(img1, img2):
    mse = F.mse_loss(img1, img2)
    if mse == 0:
        return 100
    piinput_tensorel_mainput_tensor = 1.0
    return 20 * torch.log10(piinput_tensorel_mainput_tensor / torch.sqrt(mse))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import shutil
import os

# Path to the folder in Google Drive
source_folder = '/content/drive/My Drive/LOLdataset/'

# Path where you want to copy the folder
destination_folder = '/content/LOLdataset/'

# Create target directory if it doesn't einput_tensorist
if not os.path.einput_tensorists(destination_folder):
    os.makedirs(destination_folder)

# Copy the entire folder
shutil.copytree(source_folder, destination_folder, dirs_einput_tensorist_ok=True)

print(f'Folder copied to: {destination_folder}')


In [ ]:
import torch.optim as optim

# Dataset paths
train_low_dir = '/content/LOLdataset/our485/low'
train_high_dir = '/content/LOLdataset/our485/high'
val_low_dir = '/content/LOLdataset/eval15/low'
val_high_dir = '/content/LOLdataset/eval15/high'


In [ ]:
# Create datasets and dataloaders
train_dataset = LowLightDataset(train_low_dir, train_high_dir, augmentation=augmentation)
val_dataset = LowLightDataset(val_low_dir, val_high_dir, augmentation=augmentation)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)


In [ ]:
def gaussian(window_size, sigma):
    gauss = torch.tensor([torch.einput_tensorp(torch.tensor(-(input_tensor - window_size // 2) ** 2 / float(2 * sigma ** 2))) for input_tensor in range(window_size)], dtype=torch.float32)
    return gauss / gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.einput_tensorpand(channel, 1, window_size, window_size).contiguous()
    return window

def ssim(img1, img2, window_size=11, size_average=True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel).to(img1.device)

    mu1 = F.conv2d(img1, window, pad=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, pad=window_size // 2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, pad=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, pad=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, pad=window_size // 2, groups=channel) - mu1_mu2

    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    return ssim_map.mean() if size_average else ssim_map.mean(1).mean(1).mean(1)

class SSIMLoss(nn.Module):
    def __init__(model, window_size=11, size_average=True):
        super(SSIMLoss, model).__init__()
        model.window_size = window_size
        model.size_average = size_average

    def forward(model, img1, img2):
        return 1 - ssim(img1, img2, model.window_size, model.size_average)

# Combined Loss Function
class CombinedLoss(nn.Module):
    def __init__(model):
        super(CombinedLoss, model).__init__()
        model.ssim_loss = SSIMLoss()
        model.l1_loss = nn.L1Loss()

    def forward(model, output, target):
        # L1 loss
        l1_loss = model.l1_loss(output, target)

        # SSIM loss
        ssim_loss = model.ssim_loss(output, target)

        # Gradient loss
        grad_loss = model.gradient_loss(output, target)

        # Combined loss
        total_loss = 0.1 * ssim_loss + l1_loss + grad_loss

        return total_loss

    def gradient_loss(model, output, target):
        # Compute gradients
        output_grad_input_tensor = torch.abs(output[:, :, :, :-1] - output[:, :, :, 1:])
        output_grad_y = torch.abs(output[:, :, :-1, :] - output[:, :, 1:, :])
        target_grad_input_tensor = torch.abs(target[:, :, :, :-1] - target[:, :, :, 1:])
        target_grad_y = torch.abs(target[:, :, :-1, :] - target[:, :, 1:, :])

        # Compute gradient loss
        grad_loss_input_tensor = F.l1_loss(output_grad_input_tensor, target_grad_input_tensor)
        grad_loss_y = F.l1_loss(output_grad_y, target_grad_y)

        return grad_loss_input_tensor + grad_loss_y


In [ ]:
# Initialize the model, loss function, and optimizer
model = LowLightModel()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = CombinedLoss()

In [ ]:
Epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(Epochs):
    model.train()
    train_loss = 0.0
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{Epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_psnr = 0.0
    with torch.no_grad():
        for low_img, high_img in val_loader:
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    print(f"Epoch [{epoch+1}/{Epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")


In [ ]:
new_epochs = 15  # New number of epochs to run
start_epoch = 40
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_psnr = 0.0
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

torch.save(model.state_dict(), 'model_55epochs.pth')


In [ ]:
import torch
from tqdm import tqdm

# Einput_tensorample settings
new_epochs = 15  # New number of epochs to run
start_epoch = 40
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_psnr = 0.0
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

    # Save the model after each epoch
    save_path = f'model_epoch_{epoch+1}.pth'
    torch.save(model.state_dict(), save_path)
    print(f"Model saved successfully at {save_path}")


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0004)
new_epochs = 5 # New number of epochs to run
start_epoch = 70
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    # Wrap train_loader with tqdm
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_psnr = 0.0
    # Wrap val_loader with tqdm
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

# Save the state after additional training
torch.save(model.state_dict(), 'model_75epochs.pth')


In [ ]:
import torch
from torch import optim
from tqdm import tqdm

# Setting up the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0004)

# Training parameters
new_epochs = 5  # New number of epochs to run
start_epoch = 70
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    # Wrap train_loader with tqdm for progress bar
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_psnr = 0.0
    # Wrap val_loader with tqdm for progress bar
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    # Average losses and PSNR for the epoch
    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    # Log training and validation results
    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

    # Save the model state after each epoch
    save_path = f'model_epoch_{epoch+1}.pth'
    torch.save(model.state_dict(), save_path)
    print(f"Model saved successfully at {save_path}")

# Save the final state after all additional training
final_save_path = 'model_75epochs.pth'
torch.save(model.state_dict(), final_save_path)
print(f"Final model saved successfully at {final_save_path}")


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.00002)
new_epochs = 5 # New number of epochs to run
start_epoch = 100
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    # Wrap train_loader with tqdm
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    model.eval()
    val_psnr = 0.0
    # Wrap val_loader with tqdm
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

# Save the state after additional training
torch.save(model.state_dict(), 'model_100epochs.pth')



In [ ]:
import torch
from torch import optim
from tqdm import tqdm

# Set up the optimizer with a very low learning rate
optimizer = optim.Adam(model.parameters(), lr=0.00002)

# Define training parameters
new_epochs = 5  # New number of epochs to run
start_epoch = 100
total_epochs = start_epoch + new_epochs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(start_epoch, total_epochs):
    model.train()
    train_loss = 0.0
    # Wrap train_loader with tqdm for progress feedback
    for low_img, high_img in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Training", leave=False):
        low_img, high_img = low_img.to(device), high_img.to(device)

        # Forward pass
        output = model(low_img)

        # Compute loss
        loss = criterion(output, high_img)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation phase
    model.eval()
    val_psnr = 0.0
    # Wrap val_loader with tqdm for validation progress
    with torch.no_grad():
        for low_img, high_img in tqdm(val_loader, desc=f"Epoch [{epoch+1}/{total_epochs}] Validation", leave=False):
            low_img, high_img = low_img.to(device), high_img.to(device)
            output = model(low_img)
            val_psnr += psnr(output, high_img).item()

    # Calculate average loss and PSNR for the epoch
    train_loss /= len(train_loader)
    val_psnr /= len(val_loader)

    # Log training and validation results
    print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {train_loss:.4f}, PSNR: {val_psnr:.2f} dB")

    # Save the model state after each epoch
    save_path = f'model_epoch_{epoch+1}.pth'
    torch.save(model.state_dict(), save_path)
    print(f"Model saved successfully at {save_path}")

# Save the final state after all additional training
final_save_path = 'model_105epochs.pth'
torch.save(model.state_dict(), final_save_path)
print(f"Final model saved successfully at {final_save_path}")



In [ ]:
model.eval()
augmentation = augmentations.Compose([
    augmentations.Resize((400, 600)),
    augmentations.ToTensor()
])

input_img_path = '/content/Screenshot 2024-06-05 214421.png'
input_img = Image.open(input_img_path).convert('RGB')
input_tensor = augmentation(input_img).unsqueeze(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
input_tensor = input_tensor.to(device)

with torch.no_grad():
    enhanced_tensor = model(input_tensor)

    enhanced_tensor = enhanced_tensor.cpu()
enhanced_img = augmentations.ToPILImage()(enhanced_tensor.squeeze())
plt.imshow(enhanced_img)
plt.title('Enhanced Image')
plt.ainput_tensoris('off')
plt.show()


In [ ]:
img = cv2.imread("/content/Screenshot 2024-06-05 214421.png")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.ainput_tensoris('off')
plt.show()


In [ ]:
torch.save(model.state_dict(), 'model_100epoch.pth')